In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

In [2]:
!pip install pyannote.audio

  Using cached pyannote.audio-3.3.2-py2.py3-none-any.whl.metadata (11 kB)
  Using cached asteroid_filterbanks-0.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached pyannote.database-5.1.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached pyannote.metrics-3.2.1-py3-none-any.whl.metadata (1.3 kB)
  Using cached pyannote.pipeline-3.0.1-py3-none-any.whl.metadata (897 bytes)
  Using cached speechbrain-1.0.2-py3-none-any.whl.metadata (23 kB)
  Using cached torch_audiomentations-0.11.1-py3-none-any.whl.metadata (14 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached julius-0.2.7-py3-none-any.whl
  Using cached torch_pitch_shift-1.2.5-py3-none-any.whl.metadata (2.5 kB)
Using cached pyannote.audio-3.3.2-py2.py3-none-any.whl (898 kB)
Using cached asteroid_filterbanks-0.4.0-py3-none-any.whl (29 kB)
Using cached lightning-2.4.0-py3-none-any.whl (810 kB)
Using cached pyannote.database-5.1.0-py

In [9]:
!pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [10]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the Hugging Face token
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

# Check if the token was loaded correctly
if huggingface_token:
    print("Hugging Face token loaded successfully.")
else:
    print("Failed to load Hugging Face token. Check your .env file.")

Hugging Face token loaded successfully.


In [21]:
import warnings

# Suppress specific warnings by category
warnings.filterwarnings("ignore")

In [20]:
!pip install pydub

## Load pyannote diarization model

In [54]:
from pydub import AudioSegment
from pyannote.audio import Pipeline
import os
import torch

# Replace 'your_huggingface_token' with your actual Hugging Face access token
access_token = "your_huggingface_token"

# Initialize the pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=access_token)

# Check if CUDA is available and move the pipeline to GPU if it is
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
else:
    print("CUDA is not available. The pipeline will run on the CPU.")

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../../../../home/luis/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding

## Convert to wav and diarize

In [55]:
# Path to your audio file
audio_file = "../../audio_samples/audio_sample.mp3"  # Replace with your audio file path

# Convert mp3 to wav
audio = AudioSegment.from_mp3(audio_file)
wav_file = "../../audio_samples/audio_sample.wav"  # Define the new wav file path
audio.export(wav_file, format="wav")

<_io.BufferedRandom name='../../audio_samples/audio_sample.wav'>

In [56]:
# Apply the pipeline to the wav file
diarization = pipeline(wav_file)

# Print the diarization results
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

start=1.0s stop=4.6s speaker_SPEAKER_00
start=6.0s stop=13.5s speaker_SPEAKER_00
start=14.3s stop=17.0s speaker_SPEAKER_01
start=18.7s stop=21.7s speaker_SPEAKER_01
start=23.0s stop=26.5s speaker_SPEAKER_01
start=26.5s stop=26.7s speaker_SPEAKER_00
start=27.7s stop=32.8s speaker_SPEAKER_00
start=32.8s stop=32.8s speaker_SPEAKER_01


In [57]:
# Initialize an empty list to store the diarization entries
diarization_list = []

# Apply the pipeline to the wav file
diarization = pipeline(wav_file)

# Iterate over the diarization results and build the list of dictionaries
for turn, _, speaker in diarization.itertracks(yield_label=True):
    # Create a dictionary for each diarization segment
    diarization_entry = {
        'start': turn.start,
        'stop': turn.end,
        'speaker': speaker
    }
    # Add the dictionary to the list
    diarization_list.append(diarization_entry)

diarization_list

[{'start': 0.99284375, 'stop': 4.55346875, 'speaker': 'SPEAKER_00'},
 {'start': 5.98784375, 'stop': 13.53096875, 'speaker': 'SPEAKER_00'},
 {'start': 14.30721875, 'stop': 17.04096875, 'speaker': 'SPEAKER_01'},
 {'start': 18.67784375, 'stop': 21.715343750000002, 'speaker': 'SPEAKER_01'},
 {'start': 22.980968750000002, 'stop': 26.52471875, 'speaker': 'SPEAKER_01'},
 {'start': 26.52471875, 'stop': 26.74409375, 'speaker': 'SPEAKER_00'},
 {'start': 27.70596875, 'stop': 32.81909375, 'speaker': 'SPEAKER_00'},
 {'start': 32.81909375, 'stop': 32.83596875, 'speaker': 'SPEAKER_01'}]

## Use whisper for transcription

In [45]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# Set device and data types
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Specify the model ID
model_id = "openai/whisper-large-v3-turbo"

# Load the model
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)

# Create the pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

In [48]:
# Transcribe an audio file directly in the pipeline call
audio_file = "../../audio_samples/audio_sample.mp3"  # Replace with your audio file path
result = pipe(audio_file, return_timestamps=True)
print(result["chunks"])  # Displays transcription with timestamps

[{'timestamp': (0.0, 4.04), 'text': ' English spelling does not always indicate pronunciation.'}, {'timestamp': (5.42, 13.0), 'text': ' It was famously claimed that the word fish could be spelt G-H-O-T-I because...'}, {'timestamp': (14.36, 17.44), 'text': ' G-H in enough is pronounced F.'}, {'timestamp': (18.16, 21.64), 'text': ' O in women is pronounced I.'}, {'timestamp': (22.56, 26.68), 'text': ' T-I in motion is pronounced SH.'}, {'timestamp': (0.0, 5.0), 'text': ' So, G-H-O-T-I could be pronounced...'}, {'timestamp': (5.62, 6.0), 'text': ' Fish.'}]


In [59]:
transcription = result["chunks"]
transcription

[{'timestamp': (0.0, 4.04),
  'text': ' English spelling does not always indicate pronunciation.'},
 {'timestamp': (5.42, 13.0),
  'text': ' It was famously claimed that the word fish could be spelt G-H-O-T-I because...'},
 {'timestamp': (14.36, 17.44), 'text': ' G-H in enough is pronounced F.'},
 {'timestamp': (18.16, 21.64), 'text': ' O in women is pronounced I.'},
 {'timestamp': (22.56, 26.68), 'text': ' T-I in motion is pronounced SH.'},
 {'timestamp': (0.0, 5.0), 'text': ' So, G-H-O-T-I could be pronounced...'},
 {'timestamp': (5.62, 6.0), 'text': ' Fish.'}]

## Combine models

In [60]:
from collections import defaultdict

def get_overlap(a_start, a_end, b_start, b_end):
    """Calculate the overlap duration between two time intervals."""
    overlap_start = max(a_start, b_start)
    overlap_end = min(a_end, b_end)
    return max(0.0, overlap_end - overlap_start)

def assign_speakers_to_transcription(diarization, transcription):
    """
    Assign speaker labels to transcription chunks based on diarization.

    Parameters:
    - diarization: List of dictionaries with keys 'start', 'stop', 'speaker'.
    - transcription: List of dictionaries with keys 'timestamp', 'text'.

    Returns:
    - A new list of transcription chunks with an added 'speaker' key.
    """
    # For each transcription chunk
    for chunk in transcription:
        t_start, t_end = chunk['timestamp']
        # Keep track of overlapping durations per speaker
        overlaps = defaultdict(float)
        for diar in diarization:
            d_start = diar['start']
            d_end = diar['stop']
            speaker = diar['speaker']
            overlap = get_overlap(t_start, t_end, d_start, d_end)
            if overlap > 0:
                overlaps[speaker] += overlap
        # Determine the speaker with the maximum overlap
        if overlaps:
            assigned_speaker = max(overlaps.items(), key=lambda x: x[1])[0]
        else:
            assigned_speaker = 'Unknown'
        # Assign the speaker to the chunk
        chunk['speaker'] = assigned_speaker
    return transcription

In [62]:
combined_result = assign_speakers_to_transcription(diarization_list, transcription)
combined_result

[{'timestamp': (0.0, 4.04),
  'text': ' English spelling does not always indicate pronunciation.',
  'speaker': 'SPEAKER_00'},
 {'timestamp': (5.42, 13.0),
  'text': ' It was famously claimed that the word fish could be spelt G-H-O-T-I because...',
  'speaker': 'SPEAKER_00'},
 {'timestamp': (14.36, 17.44),
  'text': ' G-H in enough is pronounced F.',
  'speaker': 'SPEAKER_01'},
 {'timestamp': (18.16, 21.64),
  'text': ' O in women is pronounced I.',
  'speaker': 'SPEAKER_01'},
 {'timestamp': (22.56, 26.68),
  'text': ' T-I in motion is pronounced SH.',
  'speaker': 'SPEAKER_01'},
 {'timestamp': (0.0, 5.0),
  'text': ' So, G-H-O-T-I could be pronounced...',
  'speaker': 'SPEAKER_00'},
 {'timestamp': (5.62, 6.0), 'text': ' Fish.', 'speaker': 'SPEAKER_00'}]

In [64]:
for chunk in combined_result:
    print(f"Speaker {chunk['speaker']}: {chunk['text']}")

Speaker SPEAKER_00:  English spelling does not always indicate pronunciation.
Speaker SPEAKER_00:  It was famously claimed that the word fish could be spelt G-H-O-T-I because...
Speaker SPEAKER_01:  G-H in enough is pronounced F.
Speaker SPEAKER_01:  O in women is pronounced I.
Speaker SPEAKER_01:  T-I in motion is pronounced SH.
Speaker SPEAKER_00:  So, G-H-O-T-I could be pronounced...
Speaker SPEAKER_00:  Fish.
